In [1]:
import numpy as np
import heapq

In [2]:
# Day 1
# Part A
calories = [0]
with open('day1.txt') as fid:
    for line in fid:
        if line[0] == '\n':
            calories.append(0)
            continue
        calories[-1] += int(line[:-1])
calories.sort()
print(calories[-1])
# Part B
print(sum(calories[-3:]))

69795
208437


In [3]:
# Day 2
# Part A
l, r = np.loadtxt('day2.txt', dtype=int, unpack=True,
                  converters={0:lambda s: {b'A':1, b'B':2, b'C':3}[s], 1:lambda s: {b'X':1, b'Y':2, b'Z':3}[s]})
print(sum((((r + 3 - l) % 3 + 1) % 3)*3 + r))
# Part B
print(sum((r - 1)*3 +  (l + r) % 3 + 1))


11767
13886


In [4]:
# Day 3
# Part A
with open('day3.txt') as fid:
    all_lines = fid.readlines()
priorities = 0
for line in all_lines:
    n = (len(line) - 1)//2
    common_letter = (set(line[:n]) & set(line[n:-1])).pop()
    priorities += ord(common_letter) - (96 if common_letter.islower() else 38)
print(priorities)
# Part B
priorities = 0
for j in range(len(all_lines)//3):
    common_letter = (set(all_lines[3*j + 0][:-1]) & set(all_lines[3*j + 1][:-1]) & set(all_lines[3*j + 2][:-1])).pop()
    priorities += ord(common_letter) - (96 if common_letter.islower() else 38)
print(priorities)
        

7824
2798


In [5]:
# Day 4
# Part A
with open('day4.txt') as fid:
    data = np.array([list(map(int, couple.split('-'))) for couple in fid.read().strip().replace(',','-').split('\n')], dtype=int)
print((((data[:,3] >= data[:,1]) & (data[:,2] <= data[:,0])) | ((data[:,3] <= data[:,1]) & (data[:,2] >= data[:,0]))).sum())
# Part B
print((np.maximum(data[:,0], data[:,2]) <= np.minimum(data[:,1], data[:,3])).sum())

602
891


In [6]:
# Day 5
# Part A
with open('day5.txt') as fid:
    stacks_ini_lines, movement = fid.read().split('\n\n')
stacks = ['']*10
for line in stacks_ini_lines.split('\n')[:-1]:
    for j in range(1, 10):
        stacks[j] += line[4*(j - 1) + 1]
stacks = [s.strip()[::-1] for s in stacks]

def get_top_crates_after_movement(stacks, movements, order=1):
    for line in movement.strip().split('\n'):
        tmp = line.split()
        n, fro, to = int(tmp[1]), int(tmp[3]), int(tmp[5])
        stacks[fro], crates_on_crane = stacks[fro][:-n], stacks[fro][-n:]
        stacks[to] += crates_on_crane[::order]
    return ''.join([stacks[j][-1] for j in range(1, 10)])

for order in [-1, 1]:
    print(get_top_crates_after_movement([s for s in stacks], movement.strip().split('\n'), order))

BSDMQFLSP
PGSQBFLDP


In [7]:
%%timeit
# Day 5
# Part A
with open('day5.txt') as fid:
    stacks_ini_lines, movement = fid.read().split('\n\n')
stacks = ['']*10
for line in stacks_ini_lines.split('\n')[:-1]:
    for j in range(1, 10):
        stacks[j] += line[4*(j - 1) + 1]
stacks = [s.strip()[::-1] for s in stacks]

def get_top_crates_after_movement(stacks, movements, order=1):
    for line in movement.strip().split('\n'):
        tmp = line.split()
        n, fro, to = int(tmp[1]), int(tmp[3]), int(tmp[5])
        stacks[fro], crates_on_crane = stacks[fro][:-n], stacks[fro][-n:]
        stacks[to] += crates_on_crane[::order]
    return ''.join([stacks[j][-1] for j in range(1, 10)])

for order in [-1, 1]:
    s = get_top_crates_after_movement([s for s in stacks], movement.strip().split('\n'), order)

1.05 ms ± 27 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
%%timeit
# Day 5
# Part A
with open('day5.txt') as fid:
    stacks_ini_lines, movement = fid.read().split('\n\n')
stacks = ['']*10
for line in stacks_ini_lines.split('\n')[:-1]:
    for j in range(1, 10):
        stacks[j] += line[4*(j - 1) + 1]
stacks = [s.strip()[::-1] for s in stacks]
stacks_np = [np.array([ord(c) for c in s], dtype=int) for s in stacks]
def get_top_crates_after_movement_np(stacks, movements, order=1):
    for line in movement.strip().split('\n'):
        tmp = line.split()
        n, fro, to = int(tmp[1]), int(tmp[3]), int(tmp[5])
        stacks[fro], crates_on_crane = stacks[fro][:-n], stacks[fro][-n:]
        stacks[to] = np.append(stacks[to],crates_on_crane[::order])
    return ''.join([chr(stacks[j][-1]) for j in range(1, 10)])
for order in [-1, 1]:
    s = get_top_crates_after_movement_np([s for s in stacks_np], movement.strip().split('\n'), order)

5.03 ms ± 189 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
import collections, copy, re

In [10]:
%%timeit
# Read in data
stacks = None
Rearrangement = collections.namedtuple(
    'Rearrangement', ('num', 'src', 'dst'),
)
procedure = []
with open('day5.txt') as f:
    # Read in stacks of crates
    for line in f:
        if stacks is None:
            stacks = [[] for _ in range(len(line)//4)]
        if not '[' in line:
            break
        for i, content in enumerate(line[1::4]):
            if content.isalpha():
                stacks[i].append(content)
    for stack in stacks:
        stack.reverse()
    # Read in rearrangements
    f.readline()
    for line in f:
        nums = list(map(int, re.search(r'(\d+).+(\d+).+(\d+)', line).groups()))
        for i in range(1, 3):
            nums[i] -= 1
        procedure.append(Rearrangement(*nums))
stacks_ori = stacks
reset_stacks = lambda: copy.deepcopy(stacks_ori)

# Part 1
stacks = reset_stacks()
for r in procedure:
    for _ in range(r.num):
        stacks[r.dst].append(stacks[r.src].pop())
get_top = lambda stacks: ''.join(stack[-1] for stack in stacks)
#print('part one:', get_top(stacks))
s = get_top(stacks)

# Part 2
stacks = reset_stacks()
for r in procedure:
    stacks[r.dst] += stacks[r.src][ -r.num:]
    stacks[r.src]  = stacks[r.src][:-r.num ]
#print('part two:', get_top(stacks))
t = get_top(stacks)

2.16 ms ± 20.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
